# Async Python

## A briefing on asynchronous python coding, essential in Agent engineering

Here is a masterful tutorial by you-know-who with exercises and comparisons.

https://chatgpt.com/share/680648b1-b0a0-8012-8449-4f90b540886c

This includes how to run async code from a python module.

### And now some examples:

In [4]:
# Let's define an async function

import asyncio

async def do_some_work():
    print("Starting work")
    await asyncio.sleep(2)
    print("Work complete")


In [2]:
# What will this do?

do_some_work()

<coroutine object do_some_work at 0x0000019F2540B880>

In [5]:
# OK let's try that again!

await do_some_work()

Starting work
Work complete


In [6]:
# What's wrong with this?

async def do_a_lot_of_work():
    do_some_work()
    do_some_work()
    do_some_work()

await do_a_lot_of_work()

C:\Users\KaranJJoshi\AppData\Local\Temp\ipykernel_1068\1833959015.py:4: RuntimeWarning: coroutine 'do_some_work' was never awaited
  do_some_work()
C:\Users\KaranJJoshi\AppData\Local\Temp\ipykernel_1068\1833959015.py:5: RuntimeWarning: coroutine 'do_some_work' was never awaited
  do_some_work()
C:\Users\KaranJJoshi\AppData\Local\Temp\ipykernel_1068\1833959015.py:6: RuntimeWarning: coroutine 'do_some_work' was never awaited
  do_some_work()


In [7]:
# Interesting warning! Let's fix it

async def do_a_lot_of_work():
    await do_some_work()
    await do_some_work()
    await do_some_work()

await do_a_lot_of_work()

Starting work
Work complete
Starting work
Work complete
Starting work
Work complete


In [8]:
# And now let's do it in parallel
# It's important to recognize that this is not "multi-threading" in the way that you may be used to
# The asyncio library is running on a single thread, but it's using a loop to switch between tasks while one is waiting

async def do_a_lot_of_work_in_parallel():
    await asyncio.gather(do_some_work(), do_some_work(), do_some_work())

await do_a_lot_of_work_in_parallel()

Starting work
Starting work
Starting work
Work complete
Work complete
Work complete


### Finally - try writing a python module that calls do_a_lot_of_work_in_parallel

See the link at the top; you'll need something like this in your module:

```python
if __name__ == "__main__":
    asyncio.run(do_a_lot_of_work_in_parallel())
```

In [13]:
import time

def blocking():
    time.sleep(1)
    print("Blocking done")

async def non_blocking():
    await asyncio.sleep(1)
    print("Async done")

start = time.time()
for i in range(3):
    blocking()
end = time.time()
print(f"Blocking took {end - start} seconds")

start = time.time()
asyncio.gather(non_blocking(), non_blocking(), non_blocking())
end = time.time()
print(f"Non-blocking took {end - start} seconds")

Blocking done
Blocking done
Blocking done
Blocking took 3.007016897201538 seconds
Non-blocking took 0.0 seconds


Async done
Async done
Async done


In [ ]:
import gradio  as gr

async def do_work(name):
    print(f"Hello : {name}")
    await asyncio.sleep(1)
    print(f"Goodbye : {name}")
    return f"Hello : {name}"

gr.Interface(fn=do_work, inputs=gr.Textbox(lines=1, placeholder="Enter your name"), outputs=gr.Textbox(lines=1, placeholder="Enter your name")).launch()

# gr.Interface(fn=do_work, inputs="text", outputs="text").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Hello : Karan
Goodbye : Karan
Created dataset file at: .gradio\flagged\dataset1.csv


In [18]:
!uv add aiohttp

Resolved 224 packages in 17.92s
Audited 210 packages in 1ms


In [23]:
import asyncio
import aiohttp
from bs4 import BeautifulSoup

async def fetch_html(session, url):
    async with session.get(url) as response:
        return await response.text()

async def extract_title(session, url):
    try:
        html = await fetch_html(session, url)
        soup = BeautifulSoup(html, 'html.parser')
        title = soup.title.string.strip() if soup.title else "No title found"
        print(f"[✔] {url} --> {title}")
    except Exception as e:
        print(f"[✖] {url} --> Error: {e}")

async def main():
    urls = [
        "https://www.bbc.com/news",
        "https://www.cnn.com",
        "https://www.nytimes.com",
        "https://techcrunch.com",
        "https://www.theverge.com",
    ]

    async with aiohttp.ClientSession() as session:
        tasks = [extract_title(session, url) for url in urls]
        await asyncio.gather(*tasks)

if __name__ == "__main__":
    await (main())


[✖] https://www.cnn.com --> Error: Cannot connect to host www.cnn.com:443 ssl:True [SSLCertVerificationError: (1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')]
[✔] https://www.bbc.com/news --> BBC News - Breaking news, video and the latest top stories from the U.S. and around the world
[✔] https://www.theverge.com --> The Verge
[✔] https://techcrunch.com --> TechCrunch | Startup and Technology News
[✔] https://www.nytimes.com --> The New York Times - Breaking News, US News, World News and Videos
